In [1]:
## 1 · Imports y rutas globales
#Carga de librerías y definición de carpetas base.
import warnings, re, math, random
from pathlib import Path
from collections import defaultdict
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import xgboost as xgb
from sklearn.metrics import log_loss, accuracy_score
import optuna

# ---- Rutas base ----
ROOT_CANDIDATES = [Path("data"), Path("/data"), Path("/mnt/data"), Path(".")]
DATA_DIR  = next((p for p in ROOT_CANDIDATES if (p / "atp_players.csv").exists()), None)
assert DATA_DIR, "No se encontraron los CSV de ATP en las rutas candidatas."

PLAYERS_CSV = DATA_DIR / "atp_players.csv"
DRAW_CSV    = DATA_DIR / "ao_2025_draw.csv"


C:\Users\ianam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## 2 · Carga y limpieza inicial de partidos ATP
def load_matches(pattern: str = "atp_matches_*.csv", roots = ROOT_CANDIDATES) -> pd.DataFrame:
    #Une todos los CSV año‑a‑año en un único DataFrame ordenado por fecha.
    dfs = []
    for root in roots:
        for fp in sorted(root.glob(pattern)):
            dfs.append(pd.read_csv(fp, parse_dates=["tourney_date"]))
    if not dfs:
        raise FileNotFoundError(f"No se hallaron archivos {pattern} en {roots}")
    df = pd.concat(dfs, ignore_index=True)
    # Sólo columnas esenciales para Elo + XGB
    keep = ["tourney_date", "surface", "winner_id", "loser_id"]
    return df[keep].sort_values("tourney_date").reset_index(drop=True)

matches = load_matches()
orig_rows = len(matches)
# Descartamos filas sin superficie declarada
matches = matches.dropna(subset=["surface"]).reset_index(drop=True)
print(f"Matches cargados: {orig_rows:,}  → tras limpiar NaNs en 'surface': {len(matches):,}")
print(matches.surface.value_counts(dropna=False).head())
f"Matches cargados: {matches.shape[0]:,}"


Matches cargados: 194,996  → tras limpiar NaNs en 'surface': 192,006
surface
Hard      78287
Clay      69418
Grass     23401
Carpet    20900
Name: count, dtype: int64


'Matches cargados: 192,006'

In [3]:
## 3 · Cálculo de Elo dinámico y sensible a superficie
def compute_elo_features(df: pd.DataFrame, *, k: int = 32, alpha: float = 0.30):
    #Devuelve un DataFrame con las columnas pre‑partido `a_elo`, `b_elo`, `delta_elo`.

    #overall[pid]**   : Elo genérico  (Hard+Clay+Grass+Carpet)
    #surface[pid][s]**: Elo específico de la superficie *s*
    #alpha**          : Peso relativo del Elo por superficie (0 = ignora superficie).

    overall   = defaultdict(lambda: 1500.0)
    surfaces  = defaultdict(lambda: {s: 1500.0 for s in ["Hard", "Clay", "Grass", "Carpet"]})
    records   = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Elo loop"):
        w, l  = int(row.winner_id), int(row.loser_id)
        surf  = (row.surface or "Hard").title()

        w_r = (1-alpha)*overall[w] + alpha*surfaces[w][surf]
        l_r = (1-alpha)*overall[l] + alpha*surfaces[l][surf]
        exp_w = 1 / (1 + 10**((l_r - w_r)/400))

        records.append({"a_elo": w_r, "b_elo": l_r, "delta_elo": w_r - l_r})

        delta = k * (1 - exp_w)
        overall[w]  += delta; overall[l]  -= delta
        surfaces[w][surf] += delta; surfaces[l][surf] -= delta

    return pd.DataFrame(records), overall, surfaces

elo_feats, final_overall, final_surface = compute_elo_features(matches)


Elo loop: 100%|██████████| 192006/192006 [00:08<00:00, 21612.96it/s]


In [4]:
## 4 · Dataset balanceado (ganador vs perdedor)
# Positivos: ganador como lado A ─────────
X_pos = elo_feats.copy()
y_pos = np.ones(len(X_pos), dtype=np.int8)

# Negativos: perdedor como lado A ────────
X_neg = X_pos.copy()
X_neg[["a_elo", "b_elo"]] = X_neg[["b_elo", "a_elo"]].values
X_neg["delta_elo"] *= -1
y_neg = np.zeros(len(X_neg), dtype=np.int8)

X_full = pd.concat([X_pos, X_neg], ignore_index=True)
y_full = np.concatenate([y_pos, y_neg])


In [5]:
## 5 · Split cronológico train / val / test
# Dos fechas por match (porque espejamos)
dates_full = pd.concat([matches.tourney_date]*2, ignore_index=True)

train = dates_full.dt.year <= 2022
val   = dates_full.dt.year == 2023
test  = dates_full.dt.year == 2024

X_train, y_train = X_full[train], y_full[train]
X_val,   y_val   = X_full[val],   y_full[val]
X_test,  y_test  = X_full[test],  y_full[test]

print("Distribución:")
for name, y in zip(["Train", "Val", "Test"], [y_train, y_val, y_test]):
    print(f" · {name}: {len(y):,} obs – Pos={y.mean():.3f}")


Distribución:
 · Train: 371,994 obs – Pos=0.500
 · Val: 5,866 obs – Pos=0.500
 · Test: 6,152 obs – Pos=0.500


In [6]:
## 6 · Búsqueda de hiperparámetros con Optuna
dtrain = xgb.DMatrix(X_train, label=y_train)
dval   = xgb.DMatrix(X_val,   label=y_val)

def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
        "alpha":  trial.suggest_float("alpha",  1e-8, 10.0, log=True),
    }
    booster = xgb.train(params, dtrain, num_boost_round=500,
                        evals=[(dval, "val")],
                        early_stopping_rounds=50, verbose_eval=False)
    return booster.best_score

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40, show_progress_bar=True)
print("Mejor pérdida log:", study.best_value, "\nParams:", study.best_params)


[I 2025-06-30 19:36:14,742] A new study created in memory with name: no-name-d5d6031d-3007-45e6-8493-9e8f208e9c61
Best trial: 0. Best value: 0.623134:   2%|▎         | 1/40 [00:04<02:42,  4.17s/it]

[I 2025-06-30 19:36:18,912] Trial 0 finished with value: 0.6231336489553505 and parameters: {'eta': 0.010577367847204571, 'max_depth': 7, 'subsample': 0.755631492651708, 'colsample_bytree': 0.6385503892015613, 'lambda': 0.011834788147411911, 'alpha': 0.0006466760806552321}. Best is trial 0 with value: 0.6231336489553505.


Best trial: 0. Best value: 0.623134:   5%|▌         | 2/40 [00:05<01:38,  2.59s/it]

[I 2025-06-30 19:36:20,392] Trial 1 finished with value: 0.6234946026293097 and parameters: {'eta': 0.017791136706963664, 'max_depth': 6, 'subsample': 0.7490979086990577, 'colsample_bytree': 0.701578082708999, 'lambda': 6.135356436009329e-05, 'alpha': 8.262089700651654e-06}. Best is trial 0 with value: 0.6231336489553505.


Best trial: 0. Best value: 0.623134:   8%|▊         | 3/40 [00:06<01:03,  1.72s/it]

...

[I 2025-06-30 19:36:45,134] Trial 37 finished with value: 0.6244280038371637 and parameters: {'eta': 0.12418702371316383, 'max_depth': 5, 'subsample': 0.8631984641543338, 'colsample_bytree': 0.8555333395050068, 'lambda': 0.020414836439049327, 'alpha': 7.367324298877198e-07}. Best is trial 23 with value: 0.6227997689639042.


Best trial: 23. Best value: 0.6228:  98%|█████████▊| 39/40 [00:32<00:00,  1.08it/s]

[I 2025-06-30 19:36:47,002] Trial 38 finished with value: 0.6230398848445808 and parameters: {'eta': 0.02186639527630752, 'max_depth': 6, 'subsample': 0.7654524776815766, 'colsample_bytree': 0.6074843815297966, 'lambda': 0.002565090684352852, 'alpha': 1.1468876276269137e-07}. Best is trial 23 with value: 0.6227997689639042.


Best trial: 23. Best value: 0.6228: 100%|██████████| 40/40 [00:32<00:00,  1.22it/s]

[I 2025-06-30 19:36:47,407] Trial 39 finished with value: 0.6248268244163782 and parameters: {'eta': 0.17774034131727173, 'max_depth': 4, 'subsample': 0.6800443272570065, 'colsample_bytree': 0.9876067045087694, 'lambda': 3.047998321747781, 'alpha': 0.0022502346612625944}. Best is trial 23 with value: 0.6227997689639042.
Mejor pérdida log: 0.6227997689639042 
Params: {'eta': 0.06288443475800931, 'max_depth': 5, 'subsample': 0.8165827644114687, 'colsample_bytree': 0.6581166516704186, 'lambda': 3.251122027728748e-07, 'alpha': 1.3572176372666346e-06}


In [7]:
## 7 · Modelo final y evaluación en test
best_params = {**study.best_params,
               "objective": "binary:logistic",
               "eval_metric": "logloss"}
model = xgb.train(best_params, xgb.DMatrix(X_train, label=y_train),
                  num_boost_round=study.best_trial.number)

y_pred = model.predict(xgb.DMatrix(X_test))
print("LogLoss test:", log_loss(y_test, y_pred))
print("Accuracy  :", accuracy_score(y_test, (y_pred>=0.5).astype(int)))


LogLoss test: 0.6386164832782223
Accuracy  : 0.642555266579974


In [8]:
## 8 · Carga del cuadro AO 2025 y utilidades
def load_draw_with_ids(path_draw: Path, *, players_csv=PLAYERS_CSV) -> pd.DataFrame:

    #Devuelve DataFrame con `player1_id`, `player2_id`.
    #Acepta columnas `PLAYER_1/PLAYER_2` con nombre+país.

    draw = pd.read_csv(path_draw)

    # ↓ Normaliza nombres y extrae IDs ↓
    def clean(name):
        return re.sub(r"\s*\([^)]*\)", "", str(name)).strip().lower()

    if {"player1_id", "player2_id"}.issubset(draw.columns):
        return draw[["player1_id", "player2_id"]].astype(int)

    # Detecta columnas de nombres (case‑insensitive)
    name_cols = [c for c in draw.columns if c.lower() in {"player1", "player_1", "player2", "player_2", "name1", "name2", "player", "player_"}]
    if len(name_cols) >= 2:
        p1, p2 = name_cols[:2]
        players = pd.read_csv(players_csv)
        name_map = {clean(f"{r.name_first} {r.name_last}"): r.player_id
                    for r in players.itertuples()}
        p1_ids = draw[p1].apply(lambda x: name_map.get(clean(x), np.nan))
        p2_ids = draw[p2].apply(lambda x: name_map.get(clean(x), np.nan))
        if p1_ids.isna().any() or p2_ids.isna().any():
            missing = draw.loc[p1_ids.isna() | p2_ids.isna(), [p1, p2]]
            raise ValueError(f"No se encontraron IDs para:\n{missing.head()}")
        return pd.DataFrame({"player1_id": p1_ids.astype(int), "player2_id": p2_ids.astype(int)})

    raise ValueError("Formato de draw no reconocido → columnas presentes:", draw.columns)

# Cargar draw listo para usar
SURFACE = "Hard"
draw_df = load_draw_with_ids(DRAW_CSV)
print("Draw cargado", draw_df.shape)


Draw cargado (64, 2)


C:\Users\ianam\AppData\Local\Temp\ipykernel_30408\3976213852.py:20: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv(players_csv)


In [9]:
## 9 · Simulación ronda por ronda
ALPHA, K = 0.30, 32
current       = draw_df.copy()
round_num     = 1
round_results = []  # lista de DF por ronda

# Etiquetas human‑friendly para cada ronda (32‑draw → ajusta si cambia)
round_labels = {
    1: "1st Round", 2: "2nd Round", 3: "3rd Round", 4: "4th Round",
    5: "QF", 6: "SF", 7: "Final"
}

while True:
    # --- Elo previo a cada match ---
    a_elo = current.player1_id.apply(lambda pid: (1-ALPHA)*final_overall.get(pid,1500)+ALPHA*final_surface.get(pid,{}).get(SURFACE,1500))
    b_elo = current.player2_id.apply(lambda pid: (1-ALPHA)*final_overall.get(pid,1500)+ALPHA*final_surface.get(pid,{}).get(SURFACE,1500))
    dmat  = xgb.DMatrix(pd.DataFrame({"a_elo":a_elo, "b_elo":b_elo, "delta_elo":a_elo-b_elo}))

    proba   = model.predict(dmat)
    winners = np.where(proba>=0.5, current.player1_id, current.player2_id)

    # Guarda resultado ronda
        # ---- Guarda resultado de la ronda ----
    round_df = current.copy()
    round_df["winner_id"] = winners
    round_df["round"]    = round_labels.get(round_num, f"R{round_num}")
    round_results.append(round_df)

    # Actualiza Elo con un K reducido (menor volatilidad que hist.)
    for pid_w, pid_l in zip(winners, np.where(proba>=0.5, current.player2_id, current.player1_id)):
        delta_k = K * (1 - 1/(1+10**(((final_overall[pid_l]-final_overall[pid_w])/400))))
        final_overall[pid_w] += delta_k; final_overall[pid_l] -= delta_k
        final_surface[pid_w][SURFACE] += delta_k; final_surface[pid_l][SURFACE] -= delta_k

    if len(winners) == 1:
        champion_id = int(winners[0]); break
    current = pd.DataFrame({"player1_id": winners[::2], "player2_id": winners[1::2]})
    round_num += 1

print(f"🏆  Predicted champion ID: {champion_id}")


🏆  Predicted champion ID: 206173


In [10]:
## 10 Reporte de cada ronda con nombres
players = pd.read_csv(PLAYERS_CSV, low_memory=False).set_index("player_id")
name = lambda pid: players.loc[pid, ["name_first", "name_last"]].str.cat(sep=" ")

for rnd_df in round_results:
    out = rnd_df.copy()
    out["player1_name"] = out.player1_id.map(name)
    out["player2_name"] = out.player2_id.map(name)
    out["winner_name" ] = out.winner_id .map(name)

    display(
        out[["player1_id", "player1_name", "player2_id", "player2_name", "winner_id", "winner_name"]]
        .style.set_caption(out["round"].iloc[0])
        .hide(axis="index")
    )


player1_id,player1_name,player2_id,player2_name,winner_id,winner_name
206173,Jannik Sinner,111797,Nicolas Jarry,206173,Jannik Sinner
209262,Tristan Schoolkate,106121,Taro Daniel,106121,Taro Daniel
106218,Marcos Giron,105870,Yannick Hanfmann,106218,Marcos Giron
144869,Tomas Martin Etcheverry,207925,Flavio Cobolli,207925,Flavio Cobolli
128034,Hubert Hurkacz,134868,Tallon Griekspoor,128034,Hubert Hurkacz
200175,Miomir Kecmanovic,105583,Dusan Lajovic,200175,Miomir Kecmanovic
126610,Matteo Berrettini,111815,Cameron Norrie,126610,Matteo Berrettini
111190,Zhizhen Zhang,208029,Holger Rune,208029,Holger Rune
126774,Stefanos Tsitsipas,210506,Alex Michelsen,126774,Stefanos Tsitsipas
210317,James Mccabe,212021,Martin Landaluce,212021,Martin Landaluce


player1_id,player1_name,player2_id,player2_name,winner_id,winner_name
206173,Jannik Sinner,106121,Taro Daniel,206173,Jannik Sinner
106218,Marcos Giron,207925,Flavio Cobolli,207925,Flavio Cobolli
128034,Hubert Hurkacz,200175,Miomir Kecmanovic,128034,Hubert Hurkacz
126610,Matteo Berrettini,208029,Holger Rune,208029,Holger Rune
126774,Stefanos Tsitsipas,212021,Martin Landaluce,126774,Stefanos Tsitsipas
209113,Gabriel Diallo,111575,Karen Khachanov,111575,Karen Khachanov
202103,Francisco Cerundolo,200267,Zizou Bergs,202103,Francisco Cerundolo
105948,Federico Coria,200282,Alex De Minaur,200282,Alex De Minaur
126203,Taylor Fritz,106432,Borna Coric,126203,Taylor Fritz
207681,Francisco Comesana,104792,Gael Monfils,104792,Gael Monfils


player1_id,player1_name,player2_id,player2_name,winner_id,winner_name
206173,Jannik Sinner,207925,Flavio Cobolli,206173,Jannik Sinner
128034,Hubert Hurkacz,208029,Holger Rune,208029,Holger Rune
126774,Stefanos Tsitsipas,111575,Karen Khachanov,111575,Karen Khachanov
202103,Francisco Cerundolo,200282,Alex De Minaur,200282,Alex De Minaur
126203,Taylor Fritz,104792,Gael Monfils,126203,Taylor Fritz
210097,Ben Shelton,133430,Denis Shapovalov,210097,Ben Shelton
126094,Andrey Rublev,126207,Frances Tiafoe,126207,Frances Tiafoe
200615,Alexei Popyrin,106421,Daniil Medvedev,106421,Daniil Medvedev
104925,Novak Djokovic,207830,Tomas Machac,104925,Novak Djokovic
208103,Jiri Lehecka,105777,Grigor Dimitrov,105777,Grigor Dimitrov


player1_id,player1_name,player2_id,player2_name,winner_id,winner_name
206173,Jannik Sinner,208029,Holger Rune,206173,Jannik Sinner
111575,Karen Khachanov,200282,Alex De Minaur,200282,Alex De Minaur
126203,Taylor Fritz,210097,Ben Shelton,126203,Taylor Fritz
126207,Frances Tiafoe,106421,Daniil Medvedev,106421,Daniil Medvedev
104925,Novak Djokovic,105777,Grigor Dimitrov,104925,Novak Djokovic
207733,Jack Draper,207989,Carlos Alcaraz,207989,Carlos Alcaraz
200000,Felix Auger Aliassime,126205,Tommy Paul,126205,Tommy Paul
200005,Ugo Humbert,100644,Alexander Zverev,100644,Alexander Zverev


player1_id,player1_name,player2_id,player2_name,winner_id,winner_name
206173,Jannik Sinner,200282,Alex De Minaur,206173,Jannik Sinner
126203,Taylor Fritz,106421,Daniil Medvedev,126203,Taylor Fritz
104925,Novak Djokovic,207989,Carlos Alcaraz,104925,Novak Djokovic
126205,Tommy Paul,100644,Alexander Zverev,100644,Alexander Zverev


player1_id,player1_name,player2_id,player2_name,winner_id,winner_name
206173,Jannik Sinner,126203,Taylor Fritz,206173,Jannik Sinner
104925,Novak Djokovic,100644,Alexander Zverev,104925,Novak Djokovic


player1_id,player1_name,player2_id,player2_name,winner_id,winner_name
206173,Jannik Sinner,104925,Novak Djokovic,206173,Jannik Sinner


In [11]:
## 11 · Validación con ao_2025_results.csv
RESULTS_CSV = DATA_DIR / "ao_2025_results.csv"
assert RESULTS_CSV.exists(), f"❌ No se encontró {RESULTS_CSV}"

real_raw = pd.read_csv(RESULTS_CSV)
players  = pd.read_csv(PLAYERS_CSV, low_memory=False)

# ---- Helper: nombre → id ----
name2id = {}
for r in players.itertuples(index=False):
    if pd.notna(r.name_first) and pd.notna(r.name_last):
        key = f"{str(r.name_first).lower()} {str(r.name_last).lower()}"
        name2id[key] = r.player_id

get_id = lambda s: name2id.get(str(s).lower().strip(), np.nan)

# ---- Normalizar real_df ----
if {"player1_id", "player2_id", "winner_id"}.issubset(real_raw.columns):
    real_df = real_raw.copy()
else:
    assert {"player1", "player2", "winner"}.issubset(real_raw.columns), "CSV debe tener columnas de IDs o de nombres"
    real_df = real_raw.copy()
    real_df["player1_id"] = real_df.player1.map(get_id)
    real_df["player2_id"] = real_df.player2.map(get_id)
    real_df["winner_id"]  = real_df.winner .map(get_id)
    real_df = real_df.dropna(subset=["player1_id","player2_id","winner_id"])

# ---- Predicciones simuladas ----
sim_df = pd.concat(round_results, ignore_index=True).rename(columns={"winner_id":"pred_winner"})

# ---- Unordered pair key ----
for df in (real_df, sim_df):
    df["pair_key"] = df.apply(lambda r: tuple(sorted([int(r.player1_id), int(r.player2_id)])), axis=1)

merge_keys = ["pair_key"]
if "round" in real_df.columns:
    merge_keys.append("round")

merged = real_df.merge(sim_df, on=merge_keys, how="inner")

coverage  = len(merged) / len(real_df)
accuracy  = (merged.pred_winner == merged.winner_id).mean()
print(f"🔍 Partidos en dataset real: {len(real_df)}")
print(f"🔗 Partidos cubiertos por simulación: {len(merged)}  ({coverage:.1%})")

print(f"🎯 Precisión (sobre los cubiertos): {accuracy:.2%}")

if "round" in merged.columns:
    acc_round = merged.groupby("round").apply(lambda g: (g.pred_winner==g.winner_id).mean())
    print("📊 Precisión por ronda:")
    display(acc_round.to_frame("accuracy"))

# --- Campeón ---
champion_real = (
    real_df.loc[real_df.get("round", pd.Series()).eq("Final"), "winner_id"].iloc[0]
    if "round" in real_df.columns and (real_df["round"] == "Final").any()
    else real_df.winner_id.iloc[-1]
)
print(f"🏆 Campeón real coincide con predicción: {champion_real == champion_id}")


🔍 Partidos en dataset real: 127
🔗 Partidos cubiertos por simulación: 93  (73.2%)
🎯 Precisión (sobre los cubiertos): 68.82%
📊 Precisión por ronda:


C:\Users\ianam\AppData\Local\Temp\ipykernel_30408\4253896096.py:49: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  acc_round = merged.groupby("round").apply(lambda g: (g.pred_winner==g.winner_id).mean())


,accuracy
round,
1st Round,0.703125
2nd Round,0.555556
3rd Round,0.750000
4th Round,1.000000
QF,1.000000
SF,0.000000


🏆 Campeón real coincide con predicción: True
